In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import os
import glob
import pickle
import shutil
import re

In [2]:
# WMT_ROOT='./wmt20-news-task-primary-submissions'
# METRICS_ROOT=f'metricsdata_19sept/newstest2020' 
WMT_ROOT='/Users/zhengxinyong/Desktop/globalfn_wmt/newstest2020'
METRICS_ROOT='/Users/zhengxinyong/Desktop/globalfn_wmt/newstest2020'

#### Copy sources and references

In [3]:
REFERENCES = f'{WMT_ROOT}/sgm/references/'
SOURCES =f'{WMT_ROOT}/sgm/sources/'
refmap={"newstest2020":"Human-A.0", 'newstestB2020':"Human-B.0", 'newstestP2020':"Human-P.0"}

os.makedirs(METRICS_ROOT, exist_ok=True) 

#remove file as it is identical to source and does not exist in the txt folder
# os.remove(f'{WMT_ROOT}/sgm/system-outputs/en-iu/newstest2020.en-iu.zlabs-nlp.49.sgm') 

shutil.copytree(f'{WMT_ROOT}/sgm',f'{METRICS_ROOT}/sgm')
shutil.copytree(f'{WMT_ROOT}/txt',f'{METRICS_ROOT}/txt')


FileExistsError: [Errno 17] File exists: '/Users/zhengxinyong/Desktop/globalfn_wmt/newstest2020/sgm'

#### Read sgml references and
1. add human translations to systems-outputs, and 
2. obtain the corresponding docid and segid for every line in the plaintext files.

In [4]:
flat_data = []

for fn in os.listdir(REFERENCES):
    if fn.startswith('.'):
        continue
    
    print(fn)
    testset = fn.split('-')[0]
    
    lp = fn.split('-')[1]
    lp = lp[:2] + '-' + lp[2:] 
    soup = BeautifulSoup(open(os.path.join(REFERENCES, fn)), 'html.parser')
    
    lineno = 1
    # parse
    for i in soup.refset:
        if i.name != 'doc':
            continue
        docid = i['docid']
        genre = i['genre']
        origlang = i['origlang']
        
        for seg in i.find_all('seg'):
            row = []
            row.append(testset)
            row.append(docid)
            row.append(genre)
            row.append(origlang)
            row.append(lp)
            
            row.append(seg['id'])
            row.append(lineno)
            row.append(seg.string)
            
            flat_data.append(row)
            lineno+=1
            
    sysid = refmap[testset]
    
    #add sgm human translations to systems-outputs
    with open( os.path.join(f'{METRICS_ROOT}/sgm/system-outputs/{lp}/newstest2020.{lp}.{sysid}.sgm'  ) , "w") as file:
        s=str(soup)
        s=s.replace('refset','tstset')
        s=s.replace('newstestB2020','newstest2020')
        s=s.replace('newstestP2020','newstest2020')
        file.write(re.sub(r'(\n)+', r'\1', s))            

newstest2020-zhen-ref.en.sgm
newstestB2020-ende-ref.de.sgm
newstest2020-enps-ref.ps.sgm
newstestP2020-ende-ref.de.sgm
newstestB2020-deen-ref.en.sgm
newstestB2020-enzh-ref.zh.sgm
newstest2020-frde-ref.de.sgm
newstest2020-iuen-ref.en.sgm
newstest2020-jaen-ref.en.sgm
newstest2020-enja-ref.ja.sgm
newstest2020-ruen-ref.en.sgm
newstest2020-encs-ref.cs.sgm
newstest2020-csen-ref.en.sgm
newstest2020-enpl-ref.pl.sgm
newstestB2020-ruen-ref.en.sgm
newstest2020-enkm-ref.km.sgm
newstest2020-enta-ref.ta.sgm
newstest2020-psen-ref.en.sgm
newstest2020-eniu-ref.iu.sgm
newstest2020-plen-ref.en.sgm
newstest2020-taen-ref.en.sgm
newstestB2020-zhen-ref.en.sgm
newstest2020-ende-ref.de.sgm
newstest2020-defr-ref.fr.sgm
newstest2020-enru-ref.ru.sgm
newstest2020-kmen-ref.en.sgm
newstest2020-enzh-ref.zh.sgm
newstest2020-deen-ref.en.sgm


In [6]:
references = pd.DataFrame(flat_data)
references.columns = ['testset', 'docid', 'genre', 'origlang', 'lp', 'segid','lineno', 'reference']
references.head()

,testset,docid,genre,origlang,lp,segid,lineno,reference
0,newstest2020,xinhua-zh-01.6565,news,zh,zh-en,1,1,State-owned enterprises and advantageous priva...
1,newstest2020,xinhua-zh-01.6565,news,zh,zh-en,2,2,"From September 26 to 27, in order to deepen an..."
2,newstest2020,xinhua-zh-01.6565,news,zh,zh-en,3,3,"Jin Zhuanglong, executive deputy director of t..."
3,newstest2020,xinhua-zh-01.6565,news,zh,zh-en,4,4,The enterprises participating in the survey in...
4,newstest2020,xinhua-zh-01.6565,news,zh,zh-en,5,5,"During the survey, representatives visited the..."


In [7]:
# add txt human translations to systems-outputs

for lp, ldf in references.groupby('lp'):
    s,t=lp.split('-')
    for tstset  in ldf.testset.unique():
        print(lp,tstset)
        srcfile =  f'{WMT_ROOT}/txt/references/{tstset}-{s}{t}-ref.{t}.txt'
        destfile = f'{METRICS_ROOT}/txt/system-outputs/{lp}/newstest2020.{s}-{t}.{refmap[tstset]}.txt'
        shutil.copyfile(srcfile, destfile)

cs-en newstest2020
de-en newstestB2020
de-en newstest2020
de-fr newstest2020
en-cs newstest2020
en-de newstestB2020
en-de newstestP2020
en-de newstest2020
en-iu newstest2020
en-ja newstest2020
en-km newstest2020
en-pl newstest2020
en-ps newstest2020
en-ru newstest2020
en-ta newstest2020
en-zh newstestB2020
en-zh newstest2020
fr-de newstest2020
iu-en newstest2020
ja-en newstest2020
km-en newstest2020
pl-en newstest2020
ps-en newstest2020
ru-en newstest2020
ru-en newstestB2020
ta-en newstest2020
zh-en newstest2020
zh-en newstestB2020


In [11]:
#create details file

os.makedirs(os.path.join(METRICS_ROOT, 'txt', 'details'), exist_ok=True) 

for lp, ldf in references.groupby('lp'):
    s = ldf[ldf.testset=='newstest2020']    
    s['testset'] = 'newstest2020'
    s = s[['lineno', 'testset', 'lp',  'docid',  'segid']] 
    s.to_csv(os.path.join(METRICS_ROOT, 'txt', 'details',f'{lp}.txt'),header = False, index =False, sep='\t')


/home/nmathur/miniconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


### Generate demo metrics for validation (in a very roundabout way)

In [8]:
#read system output files

SYSOUTS = f'{METRICS_ROOT}/sgm/system-outputs/*/*'  
flat_data = []

for fn in sorted(glob.glob(SYSOUTS)):
    basename = fn.split('/')[-1]
    print(basename)
    
    lp = fn.split('-')[1]
    lp = fn.split('/')[5]
    
    soup = BeautifulSoup(open(fn), 'html.parser')
    
    testset = soup.tstset['setid'] 
    for i in soup.tstset:
        if i.name != 'doc':
            continue
        docid = i['docid']
        genre = i['genre']
        origlang = i['origlang']
        sysid = basename.split('.')[2] + '.' + basename.split('.')[3]
        
            
        
        for seg in i.find_all('seg'):
            row = []
            row.append(testset)
            row.append(docid)
            row.append(genre)
            row.append(origlang)
            row.append(lp)
            
            row.append(sysid)
            
            row.append(seg['id'])
            row.append(seg.string)
            
            flat_data.append(row)

newstest2020.cs-en.CUNI-DocTransformer.1457.sgm
newstest2020.cs-en.CUNI-T2T-2018.1072.sgm
newstest2020.cs-en.CUNI-Transformer.1081.sgm
newstest2020.cs-en.Human-A.0.sgm
newstest2020.cs-en.OPPO.1481.sgm
newstest2020.cs-en.Online-A.1570.sgm
newstest2020.cs-en.Online-B.1586.sgm
newstest2020.cs-en.Online-G.1552.sgm
newstest2020.cs-en.Online-Z.1628.sgm
newstest2020.cs-en.PROMT_NMT.76.sgm
newstest2020.cs-en.SRPOL.1365.sgm
newstest2020.cs-en.UEDIN-CUNI.1359.sgm
newstest2020.cs-en.zlabs-nlp.1149.sgm
newstest2020.de-en.Human-A.0.sgm
newstest2020.de-en.Human-B.0.sgm
newstest2020.de-en.Huoshan_Translate.789.sgm
newstest2020.de-en.OPPO.1360.sgm
newstest2020.de-en.Online-A.1571.sgm
newstest2020.de-en.Online-B.1587.sgm
newstest2020.de-en.Online-G.1553.sgm
newstest2020.de-en.Online-Z.1629.sgm
newstest2020.de-en.PROMT_NMT.77.sgm
newstest2020.de-en.Tohoku-AIP-NTT.1442.sgm
newstest2020.de-en.UEDIN.1066.sgm
newstest2020.de-en.WMTBiomedBaseline.387.sgm
newstest2020.de-en.yolo.1052.sgm
newstest2020.de-en.zl

KeyError: 'setid'

In [13]:
system_outs = pd.DataFrame(flat_data)
system_outs.columns = ['testset', 'docid', 'genre', 'origlang', 'lp', 'sysid', 'segid', 'output']
system_outs.head() 

,testset,docid,genre,origlang,lp,sysid,segid,output
0,newstest2020,novinky.cz.121062,news,cs,newstest2020.cs-en.CUNI-DocTransformer.1457.sgm,CUNI-DocTransformer.1457,1,An unusually open and unique image of Karel Go...
1,newstest2020,novinky.cz.121062,news,cs,newstest2020.cs-en.CUNI-DocTransformer.1457.sgm,CUNI-DocTransformer.1457,2,This unique film feature by Olga Špátová will ...
2,newstest2020,novinky.cz.121062,news,cs,newstest2020.cs-en.CUNI-DocTransformer.1457.sgm,CUNI-DocTransformer.1457,3,"""In mid-September last year, Ivanka and Karel ..."
3,newstest2020,novinky.cz.121062,news,cs,newstest2020.cs-en.CUNI-DocTransformer.1457.sgm,CUNI-DocTransformer.1457,4,"Thanks to a quick agreement and agreement, the..."
4,newstest2020,novinky.cz.121062,news,cs,newstest2020.cs-en.CUNI-DocTransformer.1457.sgm,CUNI-DocTransformer.1457,5,The film is not meant to be a parade of Karel ...


In [14]:
#demo ref metrics

segscores = [] 
import numpy as np 
# <METRIC NAME>   <LANG-PAIR>   <TEST SET>   <SYSTEM>   <SYSTEM LEVEL SCORE>   <ENSEMBLE>   <AVAILABLE> 

for lp, ldf in system_outs.groupby('lp'): 
            print(lp) 
        
            s = ldf[['testset', 'lp', 'sysid', 'docid',  'segid'  ]] 
            s['refset']='newstest2020'
            
            #multiple references 
            if lp in ['de-en','en-de', 'en-zh', 'ru-en', 'zh-en']:
                b=s.copy()
                b.refset = 'newstestM2020' 
                segscores.append(b)             
                 
            if lp in ['de-en','en-de', 'en-zh', 'ru-en', 'zh-en']:
                b=s.copy()
                b.refset = 'newstestB2020' 
                b = b[(b.sysid != refmap['newstestB2020'])]
                segscores.append(b) 
     
            if lp == 'en-de' :
                p=s.copy() 
                p.refset = 'newstestP2020'  
                p = p[(p.sysid !=refmap['newstestP2020'])]
                segscores.append(p) 
                 
            s = s[(s.sysid != refmap['newstest2020'])] 
            segscores.append(s) 

ss=pd.concat(segscores)  

for metric_name in ['refmetric']:

    ss['metric'] = metric_name
    ss['score'] = np.random.random(len(ss))   
    ss = ss[['metric', 'lp', 'testset', 'refset', 'sysid', 'docid', 'segid', 'score']]
    # <METRIC NAME>   <LANG-PAIR>   <TEST SET>   <SYSTEM>   <SYSTEM LEVEL SCORE>   <ENSEMBLE>   <AVAILABLE> 

    ss.to_csv(f'./{metric_name}.seg.score',header = False, index =False, sep='\t')


    syss = ss.groupby(['metric','lp','testset', 'refset','sysid', 'docid']).score.mean()
    syss.to_csv(f'./{metric_name}.doc.score', header = False, index =True, sep='\t')
    syss = ss.groupby(['metric','lp','testset','refset', 'sysid']).score.mean()
    syss.to_csv(f'./{metric_name}.sys.score', header = False, index =True, sep='\t')


newstest2020.cs-en.CUNI-DocTransformer.1457.sgm
newstest2020.cs-en.CUNI-T2T-2018.1072.sgm


/home/nmathur/miniconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


newstest2020.cs-en.CUNI-Transformer.1081.sgm
newstest2020.cs-en.Human-A.0.sgm
newstest2020.cs-en.OPPO.1481.sgm
newstest2020.cs-en.Online-A.1570.sgm
newstest2020.cs-en.Online-B.1586.sgm
newstest2020.cs-en.Online-G.1552.sgm
newstest2020.cs-en.Online-Z.1628.sgm
newstest2020.cs-en.PROMT_NMT.76.sgm
newstest2020.cs-en.SRPOL.1365.sgm
newstest2020.cs-en.UEDIN-CUNI.1359.sgm
newstest2020.cs-en.zlabs-nlp.1149.sgm
newstest2020.de-en.Human-A.0.sgm
newstest2020.de-en.Human-B.0.sgm
newstest2020.de-en.Huoshan_Translate.789.sgm
newstest2020.de-en.OPPO.1360.sgm
newstest2020.de-en.Online-A.1571.sgm
newstest2020.de-en.Online-B.1587.sgm
newstest2020.de-en.Online-G.1553.sgm
newstest2020.de-en.Online-Z.1629.sgm
newstest2020.de-en.PROMT_NMT.77.sgm
newstest2020.de-en.Tohoku-AIP-NTT.1442.sgm
newstest2020.de-en.UEDIN.1066.sgm
newstest2020.de-en.WMTBiomedBaseline.387.sgm
newstest2020.de-en.yolo.1052.sgm
newstest2020.de-en.zlabs-nlp.1153.sgm
newstest2020.de-fr.Human-A.0.sgm
newstest2020.de-fr.Huoshan_Translate.130

newstest2020.ru-en.Online-A.1583.sgm
newstest2020.ru-en.Online-B.1603.sgm
newstest2020.ru-en.Online-G.1567.sgm
newstest2020.ru-en.Online-Z.1644.sgm
newstest2020.ru-en.PROMT_NMT.78.sgm
newstest2020.ru-en.ariel197197.512.sgm
newstest2020.ru-en.eTranslation.686.sgm
newstest2020.ru-en.zlabs-nlp.1164.sgm
newstest2020.ta-en.DCU.988.sgm
newstest2020.ta-en.Facebook_AI.1460.sgm
newstest2020.ta-en.GTCOM.1409.sgm
newstest2020.ta-en.Human-A.0.sgm
newstest2020.ta-en.Huoshan_Translate.1431.sgm
newstest2020.ta-en.Microsoft_STC_India.264.sgm
newstest2020.ta-en.NiuTrans.1540.sgm
newstest2020.ta-en.OPPO.1364.sgm
newstest2020.ta-en.Online-A.1584.sgm
newstest2020.ta-en.Online-B.1604.sgm
newstest2020.ta-en.Online-G.1568.sgm
newstest2020.ta-en.Online-Z.1645.sgm
newstest2020.ta-en.TALP_UPC.192.sgm
newstest2020.ta-en.UEDIN.384.sgm
newstest2020.ta-en.zlabs-nlp.1180.sgm
newstest2020.zh-en.DeepMind.381.sgm
newstest2020.zh-en.DiDi_NLP.401.sgm
newstest2020.zh-en.Huawei_TSC.889.sgm
newstest2020.zh-en.Human-A.0.sgm


In [15]:
#demo src metrics

segscores = [] 
import numpy as np 
# <METRIC NAME>   <LANG-PAIR>   <TEST SET>   <SYSTEM>   <SYSTEM LEVEL SCORE>   <ENSEMBLE>   <AVAILABLE> 

for lp, ldf in system_outs.groupby('lp'): 
            print(lp)
            s = ldf[['testset', 'lp', 'sysid', 'docid',  'segid'  ]]  
            s['refset']='newstest2020'
              
            segscores.append(s) 

ss=pd.concat(segscores)   

for metric_name in ['srcmetric' ]:
    ss['metric'] = metric_name
    ss['score'] = np.random.random(len(ss))   
    ss = ss[['metric', 'lp', 'testset', 'refset', 'sysid', 'docid', 'segid', 'score']]
    # <METRIC NAME>   <LANG-PAIR>   <TEST SET>   <SYSTEM>   <SYSTEM LEVEL SCORE>   <ENSEMBLE>   <AVAILABLE> 

    ss.to_csv(f'./{metric_name}.seg.score',header = False, index =False, sep='\t')


    syss = ss.groupby(['metric','lp','testset', 'refset','sysid', 'docid']).score.mean()
    syss.to_csv(f'./{metric_name}.doc.score', header = False, index =True, sep='\t')
    syss = ss.groupby(['metric','lp','testset','refset', 'sysid']).score.mean()
    syss.to_csv(f'./{metric_name}.sys.score', header = False, index =True, sep='\t')


newstest2020.cs-en.CUNI-DocTransformer.1457.sgm


/home/nmathur/miniconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


newstest2020.cs-en.CUNI-T2T-2018.1072.sgm
newstest2020.cs-en.CUNI-Transformer.1081.sgm
newstest2020.cs-en.Human-A.0.sgm
newstest2020.cs-en.OPPO.1481.sgm
newstest2020.cs-en.Online-A.1570.sgm
newstest2020.cs-en.Online-B.1586.sgm
newstest2020.cs-en.Online-G.1552.sgm
newstest2020.cs-en.Online-Z.1628.sgm
newstest2020.cs-en.PROMT_NMT.76.sgm
newstest2020.cs-en.SRPOL.1365.sgm
newstest2020.cs-en.UEDIN-CUNI.1359.sgm
newstest2020.cs-en.zlabs-nlp.1149.sgm
newstest2020.de-en.Human-A.0.sgm
newstest2020.de-en.Human-B.0.sgm
newstest2020.de-en.Huoshan_Translate.789.sgm
newstest2020.de-en.OPPO.1360.sgm
newstest2020.de-en.Online-A.1571.sgm
newstest2020.de-en.Online-B.1587.sgm
newstest2020.de-en.Online-G.1553.sgm
newstest2020.de-en.Online-Z.1629.sgm
newstest2020.de-en.PROMT_NMT.77.sgm
newstest2020.de-en.Tohoku-AIP-NTT.1442.sgm
newstest2020.de-en.UEDIN.1066.sgm
newstest2020.de-en.WMTBiomedBaseline.387.sgm
newstest2020.de-en.yolo.1052.sgm
newstest2020.de-en.zlabs-nlp.1153.sgm
newstest2020.de-fr.Human-A.0.sg

newstest2020.ta-en.DCU.988.sgm
newstest2020.ta-en.Facebook_AI.1460.sgm
newstest2020.ta-en.GTCOM.1409.sgm
newstest2020.ta-en.Human-A.0.sgm
newstest2020.ta-en.Huoshan_Translate.1431.sgm
newstest2020.ta-en.Microsoft_STC_India.264.sgm
newstest2020.ta-en.NiuTrans.1540.sgm
newstest2020.ta-en.OPPO.1364.sgm
newstest2020.ta-en.Online-A.1584.sgm
newstest2020.ta-en.Online-B.1604.sgm
newstest2020.ta-en.Online-G.1568.sgm
newstest2020.ta-en.Online-Z.1645.sgm
newstest2020.ta-en.TALP_UPC.192.sgm
newstest2020.ta-en.UEDIN.384.sgm
newstest2020.ta-en.zlabs-nlp.1180.sgm
newstest2020.zh-en.DeepMind.381.sgm
newstest2020.zh-en.DiDi_NLP.401.sgm
newstest2020.zh-en.Huawei_TSC.889.sgm
newstest2020.zh-en.Human-A.0.sgm
newstest2020.zh-en.Human-B.0.sgm
newstest2020.zh-en.Huoshan_Translate.919.sgm
newstest2020.zh-en.OPPO.1422.sgm
newstest2020.zh-en.Online-A.1585.sgm
newstest2020.zh-en.Online-B.1605.sgm
newstest2020.zh-en.Online-G.1569.sgm
newstest2020.zh-en.Online-Z.1646.sgm
newstest2020.zh-en.SJTU-NICT.320.sgm
newst